# Modelo Final
## Random Forest

Modelo final utilizando el set de datos modificado (incluyendo las nuevas columnas, y elimimando otras). El modelo utiliza 20 estimadores.

In [ ]:
import pandas as pd
import numpy as np
names = ['hora', 'valor_trx', 'class', 'seg_str', 'ocupacion', 'tipo_vivienda', 'nivel_academico', 'estado_civil', 'genero', 'edad', 'ingreso_rango', 'ultimas_trx', 'ultimo_mes', 'mes', 'dia']
df = pd.read_csv('set5.csv', names=names)

In [ ]:
#print(df.head(10))
print('Filas: {}\nColumnas: {}\n'.format(df.shape[0], df.shape[1]))
print(df.isnull().sum())

In [ ]:
df_t10 = df.dropna(thresh=10)
print('>> thresh 10 <<\n   Filas: {}\n   Columnas: {}\n'.format(df_t10.shape[0], df_t10.shape[1]))
df_t11 = df.dropna(thresh=11)
print('>> thresh 11 <<\n   Filas: {}\n   Columnas: {}\n'.format(df_t11.shape[0], df_t11.shape[1]))
df_t12 = df.dropna(thresh=12)
print('>> thresh 12 <<\n   Filas: {}\n   Columnas: {}\n'.format(df_t12.shape[0], df_t12.shape[1]))
df_t13 = df.dropna(thresh=13)
print('>> thresh 13 <<\n   Filas: {}\n   Columnas: {}\n'.format(df_t13.shape[0], df_t13.shape[1]))
df_t14 = df.dropna(thresh=14)
print('>> thresh 14 <<\n   Filas: {}\n   Columnas: {}\n'.format(df_t14.shape[0], df_t14.shape[1]))

In [ ]:
del df_t10
del df_t11
del df_t12
del df_t14

In [ ]:
df_t13 = df.dropna(thresh=13)
print('>> thresh 13 <<\n   Filas: {}\n   Columnas: {}\n'.format(df_t13.shape[0], df_t13.shape[1]))

In [ ]:
df1 = df_t13.fillna(value=df_t13.mode().iloc[0])
print('Filas: {}\nColumnas: {}\n'.format(df1.shape[0], df1.shape[1]))
print(df1.isnull().sum())

In [ ]:
del df_t13
del df

In [ ]:
from sklearn.model_selection import train_test_split

x = df1.drop('class', axis = 1)
y = df1['class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=548, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from functools import reduce

clf = RandomForestClassifier(n_jobs=3, n_estimators=25, random_state=895)
clf.fit(x_train, y_train)
print(' '.join(['{0:.2f}'.format(x) for x in clf.feature_importances_]))

predict = clf.predict(x_test)

In [ ]:
import scikitplot as skplt
from sklearn import metrics

print("\n#### Accuracy DT ####")
print(metrics.accuracy_score(y_test, predict))
print("\n#### Matriz de Confusion DT ####")
print(metrics.confusion_matrix(y_test, predict))  
print("\n#### Metricas DT ####")
print(metrics.classification_report(y_test, predict)) 

In [ ]:
plot = skplt.metrics.plot_confusion_matrix(y_test, predict, normalize=True, title='', figsize=(20,8))

In [ ]:
x_train_set = list()
y_train_set = list()
for g, df in x_train.groupby(np.arange(len(x_train)) // 400000):
    x_train_set.append(df)
    #print(df.shape)
    
for g, df in y_train.groupby(np.arange(len(y_train)) // 400000):
    y_train_set.append(df)
    #print(df.shape)
    
print(len(x_train_set))

In [ ]:
del x_train
del y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from functools import reduce

clf_set = list()
n_it = 1
for x_train, y_train in zip(x_train_set, y_train_set):
    clf = RandomForestClassifier(n_jobs=3, random_state=895)
    clf.fit(x_train, y_train)
    print('Iteration #{0:2d}'.format(n_it))
    print('rf score: {0:.4f}'.format(clf.score(x_test, y_test)))
    print(' '.join(['{0:.2f}'.format(x) for x in clf.feature_importances_]))
    clf_set.append(clf)
    n_it += 1

def combine_rfs(rf_a, rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

rf_combined = reduce(combine_rfs, clf_set)
# the combined model scores better than *most* of the component models
print('rf combined score: {0:.4f}'.format(rf_combined.score(x_test, y_test)))

In [ ]:
predict = rf_combined.predict(x_test)

In [ ]:
import scikitplot as skplt
from sklearn import metrics

print("\n#### Accuracy DT ####")
print(metrics.accuracy_score(y_test, predict))
print("\n#### Matriz de Confusion DT ####")
print(metrics.confusion_matrix(y_test, predict))  
print("\n#### Metricas DT ####")
print(metrics.classification_report(y_test, predict)) 

In [ ]:
plot = skplt.metrics.plot_confusion_matrix(y_test, predict, normalize=True, title='', figsize=(20,8))